# Лабораторная работая №4
## Применение логистической регрессии для решения задачи бинарной классификации
### Гончаров Ян

#### Задание 1. Выполнить предобработку данных с помощью Pandas

1.1.	 Удалить все строки данных содержащие пропуски.
1.2.	 Удалить столбцы содержащие не числовые значения, за исключением столбцов Sex и Embarked
1.3.	 Перекодировать данные в числовой вид в столбцах Sex и Embarked. Sex – 0 и 1, Embarked – 1,2,3
1.4.	 Удалить столбец PassengerId как не несущий полезной информации.
1.5.	 Вычислить процент потерянных данных после выполнения пп 1.1 – 1.2


#### Задание 2. Машинное обучение

2.1. Разбить полученные данные на обучающую и тестовые выборки
2.2. Решить задачу классификации методом логистической регрессии
2.3. Вывести точность полученной модели
2.4. Оценить влияние признака Embarked на точность модели

### Выполним предобработку данных с помощью Pandas

In [52]:
import pandas as pd

data = pd.read_csv("Titanic.csv")
size_before = data.size

# Удалим столбцы содержащие не числовые значения, за исключением столбцов Sex и Embarked
# А так же удалим столбцы не несущие полезной информации
data.drop(labels = ['PassengerId', 'Name', 'Ticket', 'Cabin'], axis = 1, inplace= True)

# Удалим все строки данных содержащие пропуски
data = data.dropna()
size_after = data.size

# Вычислим процент потерянных данных
print("Процент утерянных данных: ", 100 - round(((size_after / size_before) * 100), 3), "%")

# Перекодируем данные в числовой вид в столбцах Sex и Embarked. Sex – 0 и 1, Embarked – 1,2,3
data['Sex'] = data['Sex'].replace(['male', 'female'], [0, 1])
data['Embarked'] = data['Embarked'].replace(['C', 'Q', 'S'], [0, 1, 2])
data

Процент утерянных данных:  46.727 %


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,2
1,1,1,1,38.0,1,0,71.2833,0
2,1,3,1,26.0,0,0,7.9250,2
3,1,1,1,35.0,1,0,53.1000,2
4,0,3,0,35.0,0,0,8.0500,2
...,...,...,...,...,...,...,...,...
885,0,3,1,39.0,0,5,29.1250,1
886,0,2,0,27.0,0,0,13.0000,2
887,1,1,1,19.0,0,0,30.0000,2
889,1,1,0,26.0,0,0,30.0000,0


### Машинное обучение

In [53]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X = data.drop(labels='Survived', axis = 1)
y = data['Survived']

# Разобьём полученные данные на обучающую и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# Решим задачу классификации методом логистической регрессии
regressor = LogisticRegression(random_state=0, max_iter=1000)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

res = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(res)

# Выведем точность полученной модели
print("Оценка точности:", accuracy_score(y_test, y_pred))

     Actual  Predicted
423       0          0
178       0          0
305       1          1
292       0          0
592       0          0
..      ...        ...
267       1          0
503       0          0
100       0          1
268       1          1
624       0          0

[178 rows x 2 columns]
Оценка точности: 0.7921348314606742


### Проведём ту же классификацию, но без столбца Embarance

In [54]:
X = data.drop(labels=['Survived', 'Embarked'], axis = 1)
y = data['Survived']

# Разобьём полученные данные на обучающую и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# Решим задачу классификации методом логистической регрессии
regressor = LogisticRegression(random_state=0, max_iter=1000)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

res = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(res)

# Выведем точность полученной модели
print("Оценка точности:", accuracy_score(y_test, y_pred))

     Actual  Predicted
423       0          0
178       0          0
305       1          1
292       0          0
592       0          0
..      ...        ...
267       1          0
503       0          1
100       0          1
268       1          1
624       0          0

[178 rows x 2 columns]
Оценка точности: 0.7808988764044944


Оценка получилась немного хуже, но в районе погрешности

Используем функцию permutation_importance из библиотеки sklearn чтобы проверить важность этого признака для нашей модели

Это функция будет перемешивать столбцы признаков и оценивать насколько хуже стали оценки по сравнению с нормально обученной моделью. Если точность изменилась не сильно, значит и модель практически не опирается на этот признак.

In [55]:
from sklearn.inspection import permutation_importance

X = data.drop(labels='Survived', axis = 1)
y = data['Survived']

# Разобьём полученные данные на обучающую и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# Решим задачу классификации методом логистической регрессии
regressor = LogisticRegression(random_state=0, max_iter=1000)
regressor.fit(X_train, y_train)

result = permutation_importance(regressor, X, y, n_repeats=10, random_state=0, scoring='accuracy')

result

{'importances_mean': array([0.08651685, 0.19691011, 0.0369382 , 0.00800562, 0.00407303,
        0.00252809, 0.00140449]),
 'importances_std': array([0.01231158, 0.01675768, 0.00761671, 0.00421582, 0.0015952 ,
        0.00224719, 0.0030123 ]),
 'importances': array([[ 0.0997191 ,  0.09129213,  0.08988764,  0.08707865,  0.10955056,
          0.0744382 ,  0.08567416,  0.06460674,  0.08848315,  0.0744382 ],
        [ 0.19382022,  0.18117978,  0.1755618 ,  0.20505618,  0.19662921,
          0.22050562,  0.1741573 ,  0.2261236 ,  0.19101124,  0.20505618],
        [ 0.03511236,  0.03230337,  0.03792135,  0.01825843,  0.03651685,
          0.04073034,  0.04213483,  0.04353933,  0.03511236,  0.04775281],
        [ 0.00842697,  0.00421348,  0.01825843,  0.00702247,  0.00842697,
          0.00983146,  0.00561798,  0.00983146,  0.00702247,  0.00140449],
        [ 0.00421348,  0.00421348,  0.00421348,  0.00561798,  0.00140449,
          0.00140449,  0.00561798,  0.00561798,  0.00280899,  0.00561798

Откуда видно, что признак Embarked не является значимым для нашей модели